In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to PostgreSQL
db_uri = {db_uri}
engine = create_engine(db_uri)

# Query post details and influencer followers
query = """
SELECT p.influencer_id, SUM(p.likes) AS total_likes, SUM(p.comments) AS total_comments, 
       i.followers 
FROM post_details p
JOIN influencers i ON p.influencer_id = i.influencer_id
GROUP BY p.influencer_id, i.followers;
"""
df_engagement = pd.read_sql(query, engine)

# Calculate Engagement Rate
df_engagement['engagement_rate'] = (df_engagement['total_likes'] + df_engagement['total_comments']) / df_engagement['followers']
df_engagement = df_engagement[['influencer_id', 'engagement_rate']]

df_engagement.head()  # Preview

,influencer_id,engagement_rate
0,3850,0.881313
1,214,2.863551
2,14733,0.408523
3,21497,1.235998
4,6517,0.364008


In [3]:
df_engagement = pd.read_sql(query, engine)
print(df_engagement.columns)  # Ensure 'total_likes' exists

Index(['influencer_id', 'total_likes', 'total_comments', 'followers'], dtype='object')


In [4]:
from sklearn.ensemble import IsolationForest

# Compute Likes-to-Followers Ratio
df_engagement['likes_followers_ratio'] = df_engagement['total_likes'] / df_engagement['followers']
# Train Isolation Forest Model
model = IsolationForest(contamination=0.05, random_state=42)
df_engagement['fake_engagement_score'] = model.fit_predict(df_engagement[['likes_followers_ratio']])

# Convert -1 (outlier) to high fake score, 1 to low fake score
df_engagement['fake_engagement_score'] = df_engagement['fake_engagement_score'].apply(lambda x: 1 if x == -1 else 0)

df_engagement[['influencer_id', 'fake_engagement_score']].head()

,influencer_id,fake_engagement_score
0,3850,0
1,214,0
2,14733,0
3,21497,0
4,6517,0


In [5]:
# Fetch timestamps and engagement metrics
query = """
SELECT influencer_id, likes, comments, timestamp
FROM post_details;
"""
df_posts = pd.read_sql(query, engine)

# Normalize timestamp to days
df_posts['timestamp'] = pd.to_datetime(df_posts['timestamp'], unit='s')
df_posts['days_since_post'] = (df_posts['timestamp'].max() - df_posts['timestamp']).dt.days + 1

# Compute Virality Score = Engagement per Day
df_posts['virality_score'] = (df_posts['likes'] + df_posts['comments']) / df_posts['days_since_post']

# Aggregate per influencer
df_virality = df_posts.groupby('influencer_id')['virality_score'].mean().reset_index()

df_virality.head()

,influencer_id,virality_score
0,1,16.981010
1,2,2.321631
2,3,1.933739
3,4,2.235163
4,5,1.070621


In [8]:
print(df_engagement.columns)

Index(['influencer_id', 'total_likes', 'total_comments', 'followers',
       'likes_followers_ratio', 'fake_engagement_score'],
      dtype='object')


In [9]:
df_engagement['engagement_rate'] = (df_engagement['total_likes'] + df_engagement['total_comments']) / df_engagement['followers']
print(df_engagement.head())  # Check if engagement_rate is present

   influencer_id  total_likes  total_comments  followers  \
0           3850    4205296.0         94088.0    4878384   
1            214   25251224.0        215786.0    8893508   
2          14733      11020.0           906.0      29193   
3          21497      44880.0           758.0      36924   
4           6517      68722.0          2842.0     196600   

   likes_followers_ratio  fake_engagement_score  engagement_rate  
0               0.862026                      0         0.881313  
1               2.839287                      0         2.863551  
2               0.377488                      0         0.408523  
3               1.215470                      0         1.235998  
4               0.349552                      0         0.364008  


In [10]:
df_engagement['engagement_rate'] = (df_engagement['total_likes'] + df_engagement['total_comments']) / df_engagement['followers'].replace(0, 1)

In [11]:
print(df_engagement.columns)

Index(['influencer_id', 'total_likes', 'total_comments', 'followers',
       'likes_followers_ratio', 'fake_engagement_score', 'engagement_rate'],
      dtype='object')


In [12]:
df_final = df_engagement[['influencer_id', 'engagement_rate', 'fake_engagement_score']].merge(
    df_virality[['influencer_id', 'virality_score']], on='influencer_id', how='left'
)
df_final.head()

,influencer_id,engagement_rate,fake_engagement_score,virality_score
0,3850,0.881313,0,820.567640
1,214,2.863551,0,226.648694
2,14733,0.408523,0,2.147640
3,21497,1.235998,0,2.739521
4,6517,0.364008,0,4.105595


In [13]:
df_final.to_sql('influencer_analysis', engine, if_exists='replace', index=False)

822

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Database Connection
db_uri = {db_uri}
engine = create_engine(db_uri)

# Filters (Can be modified based on user input)
category_filter = "Fashion"
min_followers = 10000
max_followers = 500000
min_engagement_rate = 1.5  # 1.5%
max_fake_engagement_score = 20  # Lower is better
cross_niche_limit = 3  # Max number of cross-category influencers

# Combined SQL Query to Fetch Everything
query = f"""
WITH FilteredInfluencers AS (
    SELECT 
        i.influencer_id, i.username, i.category, i.followers, 
        i.posts, i.following, 
        a.engagement_rate, a.virality_score, r.pagerank_score, a.fraud_score
    FROM influencers i
    JOIN influencer_analysis a ON i.influencer_id = a.influencer_id
    JOIN influencer_rankings r ON i.influencer_id = r.influencer_id
    WHERE 
        i.category = 'Fashion' 
        AND i.followers BETWEEN 10000 AND 500000
        AND a.engagement_rate >= 1.5
        AND a.fraud_score <= 20
    ORDER BY r.pagerank_score DESC
    LIMIT 15
)
SELECT 
    f.*, 
    COALESCE(p.brandtags, 'No Collaborations') AS brand_collaborations
FROM FilteredInfluencers f
LEFT JOIN post_details p ON f.influencer_id = p.influencer_id;
"""

# Execute Query
df_results = pd.read_sql(query, engine)

# Display Results
df_results.head()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column r.pagerank_score does not exist
LINE 6:         a.engagement_rate, a.virality_score, r.pagerank_scor...
                                                     ^

[SQL: 
WITH FilteredInfluencers AS (
    SELECT 
        i.influencer_id, i.username, i.category, i.followers, 
        i.posts, i.following, 
        a.engagement_rate, a.virality_score, r.pagerank_score, a.fraud_score
    FROM influencers i
    JOIN influencer_analysis a ON i.influencer_id = a.influencer_id
    JOIN influencer_rankings r ON i.influencer_id = r.influencer_id
    WHERE 
        i.category = 'Fashion' 
        AND i.followers BETWEEN 10000 AND 500000
        AND a.engagement_rate >= 1.5
        AND a.fraud_score <= 20
    ORDER BY r.pagerank_score DESC
    LIMIT 15
)
SELECT 
    f.*, 
    COALESCE(p.brandtags, 'No Collaborations') AS brand_collaborations
FROM FilteredInfluencers f
LEFT JOIN post_details p ON f.influencer_id = p.influencer_id;
]
(Backgroun